In [2]:
from tools import *
import glob
import os
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
def get_all_trees_randomly(forest, random_state=None):

    """
    Funkcja zwraca losowe negatywne (niepoprawne) drzewo z lasu forest.

    Gdy las sklada sie tylko z jednego drzewa (poprawnego) to zwracana jest wartosc None.

    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """

    _check_sentence(forest,"forest")


    n = number_of_trees_in_forest(forest)

    if n == 1:
        Warning("There is only one tree in the forest")
        return None

    else:
        trees = [get_positive_tree(forest)]
        ids = set("-".join(sorted([node.attrib["nid"] for node in trees[0].findall('node')])))
        while n:
            tree = get_random_tree(forest,random_state)
            key = "-".join(sorted([node.attrib["nid"] for node in tree.findall('node')]))
            if id not in ids:
                trees.append(tree)
                ids.add(key)
                n -= 1

    return trees

In [4]:
def scores(positive_tree, selected_tree):
    """
    Ta funkcja powinna byc w osobnym module pt "narzedzia do ewaluacji modeli.
    
    wszystkie inne do przetwarzania drzew powinny byc w module pt "ptrzetwarzanie danych"
    """
    gold_nodes = set([node.attrib["nid"] for node in positive_tree.findall('node[@chosen="true"]')])
    selected_nodes = set([node.attrib["nid"] for node in selected_tree.findall('node')])
    intersection = set.intersection(gold_nodes, selected_nodes)
    
    precision = len(intersection)/len(selected_nodes)
    recall = len(intersection)/len(gold_nodes)
    f1 = 2/(1/precision+1/recall)
    return precision, recall, f1

In [2]:
"""
TAK ZOSTALY STWORZONE DANE UCZACE DLA SIECI

"""

data_folder = "../Składnica-frazowa-171220/**/*.xml"


j = 1
trees = []
for filename in list(glob.iglob(data_folder, recursive=True)):
    
    if j% 100 == 0:
        print(j)
    j = j + 1
    
    forest = ET.parse(filename)
 
    try:
        num_trees = number_of_trees_in_forest(forest)
        
        
        if num_trees<100000:

            trees.append(get_positive_tree(forest))
        
            
            
            

            if num_trees < 20:
                trees.append(get_random_negative_tree(forest))
                trees.append(get_random_negative_tree(forest))
            elif num_trees<50:
                for i in range(5):
                    trees.append(get_random_negative_tree(forest))
            else:
                for i in range(10):
                    trees.append(get_random_negative_tree(forest))
               
        
    except:
        pass
    
    
        
print("Liczba drzew: ",len(trees))




100
Liczba drzew:  324


In [5]:
w2vecs_full = pickle.load(open("embeddings/w2v_allwiki_nkjpfull_300.pkl","rb"),encoding="latin1")

datasets = ["Data/tokens.txt"]

words = set()
for dataset in datasets:
    with open(dataset) as f:
        for x in f.read().split():
            words.add(x)
            
words2ids_filtered = {}
vectors_filtered = []

i = 0
for word in w2vecs_full.index2word:
    if word in words:
        words2ids_filtered[word] = i
        vectors_filtered.append(w2vecs_full[word])
        i = i + 1
        
        
t = open("Data/tokens.txt","r")
tokens = [x.split() for x in t.readlines()]
t.close()
r = open("Data/rules.txt","r")
rules = [x.split() for x in r.readlines()]
r.close()


tokens2 = [[]]*len(tokens)
unique_rules = set()
for i in range(len(tokens)):
    for j in range(len(tokens[i])):
        if tokens[i][j][:2] == "__":
            tokens2[i].append(tokens[i][j]+rules[i][j])
            unique_rules.add(tokens[i][j]+rules[i][j])
        else:
            tokens2[i].append(tokens[i][j])
            
            
for i, x in zip(range(len(vectors_filtered), len(vectors_filtered) + len(unique_rules)), unique_rules):
    
    words2ids_filtered[x] = i
    vectors_filtered.append(np.random.uniform(-0.1,0.1,300))
    
vectors_filtered.append(np.random.uniform(-0.1,0.1,300))
vectors_filtered = np.array(vectors_filtered)


w2vecs = {}
w2vecs["words2ids"] = words2ids_filtered
w2vecs["vectors"] = vectors_filtered

pickle.dump(w2vecs,open("embeddings/filtered_w2v_allwiki_nkjpfull_300.pkl","wb"))

In [6]:
def words_in_from_down_to_top_order(sentence_tree):
    #print sentence_tree
    levels = np.setdiff1d(range(len(sentence_tree)),np.unique(sentence_tree)) # - zwraca slowo/a, ktore nie jest niczyim dzieckiem - czyli powinno/y byc korzeniem/korzeniami frazy/fraz
    if len(levels) == 0: # wczesniej bylo != 1, co oznaczalo, ze jezeli okazuje sie jest wiecej niz jeden korzec (lub nie ma korzenia) to zwracamy None, aby pozniej rozpoznac takie zdanie i je wywalic. Ale jak robimy batche to musi byc kilka korzeni
        return None, None
    levels = levels.tolist() 

    for i in range(len(sentence_tree)):
        x = np.setdiff1d(sentence_tree[levels[i]],-1)
        levels.extend(x[x<len(sentence_tree)])
            
    ordered_words = np.array(levels)[levels != np.array(-1)][::-1] #odwracamy kolejnosc na poczatku beda slowa znajdujace sie najglebiej
    
    order = np.zeros(len(sentence_tree),dtype='int')
    for i in range(len(sentence_tree)):
        order[ordered_words[i]] = i

    return ordered_words, order


def load_stanford_data4(labels, parents, tokens, rules, words2ids, use_batch, batch_size, nb_classes):



    sentences = []

    l = open(labels, "r")
    # 5 klas: labels = [[2 if y=='#' else int(y)+2 for y in x.split()] for x in l.readlines()] 

    # Na ten moment przyjmujemy wartosc "2" w miejsce "#"

    labels = [[int(y) for y in x.split()] for x in l.readlines()] 
    l.close()

    p = open(parents,"r")
    parents = [[int(y) for y in x.split()] for x in p.readlines()]
    p.close()

    t = open(tokens,"r")
    tokens = [x.split() for x in t.readlines()]
    t.close()
    
    r = open(rules,"r")
    rules = [x.split() for x in r.readlines()]
    r.close()
    
    k = 0
    sentence_length = 0
    current_batch, batch_tokens, batch_children_ids, batch_children_positions, batch_labels = [], [], [], [], []
    batch_words = []
    
    for labels_i, parents_i, tokens_i, rules_i in zip(labels,parents,tokens,rules):
        
        k = k + 1
         
        s = []
        for i in range(len(tokens_i)):
            
            if tokens_i[i] == "__nonterminal__":
                token = "__nonterminal__"+rules_i[i]
            else:
                token = tokens_i[i]
            
            
            s.append([i,int(parents_i[i]),labels_i[i],token])

        if len(s) == 1 and use_batch == False: #przypadek gdy fraza sklada sie z jednego tokena

            #if nb_classes == 2:
            #    if s[0][-1] < 0:
            #        continue

            sentences.append([\
                                  np.array([words2ids.get(tokens[0], -1)]),\
                                  #wyrzucamy macierz id dzieci numpy.array([-1], ndmin=2),\
                                  np.array([-1], ndmin=2), \
                                  np.array(labels_i[0]) \
                                  #,numpy.array([0])
                              ])    
                                
        else: 

            for i in range(len(s)): # nie wiem czy sie nie wywali dla frazy dlugosci 1
                children = []
                for j in range(len(s)):
                    if s[j][1] == i+1:
                        children.append(s[j][0])
                s[i].append(children)

            words = [x[0] for x in s]
            children = seq.pad_sequences([x[4] for x in s], padding='post', value = -1)
            tokens = [x[3] for x in s]
            labels_in_batch = [x[2] for x in s]
            
            ordered_words, order = words_in_from_down_to_top_order(children)

            if ordered_words is None: 
                continue

            current_sentence = [
                                  np.array([words2ids.get(x,-1) for x in tokens])[ordered_words],
                                  #wyrzucamy macierz id dzieci numpy.array([[words2ids.get(tokens[w],-1) if w>=0 else -1 for w in x] 
                                  #             for x in children[ordered_words]]), 
                                  np.array([[order[w] if w>= 0 else -1 for w in x] for x in children[ordered_words]]), 
                                  np.array(labels_in_batch)[ordered_words] 
                                  ,np.array(words)
                                  ]
            #if nb_classes == 2:
            #    if current_sentence[3][-1] <0:
            #        continue

            if use_batch == True:
                
                # w tej chwili len(current_sentence[0]) nie jest nigdzie wykorzystywane
                current_batch.append((current_sentence, len(current_sentence[0])))
                
                if len(current_batch) % batch_size == 0:

                    shift = 0
                    
                    for sent in range(batch_size):
                        
                        ##if sent > 0:
                        ##    shift = shift + current_batch[sent-1][1]
                        
                        for tok in range(len(current_batch[sent][0][0])):
                            
                            if sent == 0:
                                batch_children_positions.append(current_batch[sent][0][1][tok])
                            else:
                                batch_children_positions.append([chd+shift if chd>=0 else -1 for chd in current_batch[sent][0][1][tok]])
                            #batch_children_positions.append(current_batch[sent][0][2][tok])

                            batch_tokens.append(current_batch[sent][0][0][tok])   
                            #wyrzucamy macierz id dzieci batch_children_ids.append(current_batch[sent][0][1][tok])
                            batch_labels.append(current_batch[sent][0][2][tok])
                            batch_words.append(current_batch[sent][0][3][tok])
                                                               
                    #wyrzucamy macierz id dzieci batch_children_ids = seq.pad_sequences(batch_children_ids, padding='post', value = -1)
                    batch_children_positions = seq.pad_sequences(batch_children_positions, padding='post', value = -1)
                    
                    sentences.append([
                                        np.array(batch_tokens), 
                                        #wyrzucamy macierz id dzieci numpy.array(batch_children_ids), 
                                        np.array(batch_children_positions), 
                                        np.array(batch_labels)
                                        ,np.array(batch_words)
                                    ])
                    
                    current_batch, batch_tokens, batch_children_positions, batch_labels = [], [], [], []
                    batch_words = []
                                       
            else:
                
                sentences.append(current_sentence)

            
    # gdy liczba zdan nie jest wilokrotnosci licznosci batch, to na koncu trzeba dodac pozostale zdania:
    if use_batch == True and len(current_batch) > 0:
        
        shift = 0

        for sent in range(len(current_batch)):
 
            #if sent > 0:
            #    shift = shift + current_batch[sent-1][1]

            for tok in range(len(current_batch[sent][0][0])):

                if sent == 0:
                    batch_children_positions.append(current_batch[sent][0][1][tok])
                else:
                    batch_children_positions.append([chd+shift if chd>=0 else -1 for chd in current_batch[sent][0][1][tok]])
                #batch_children_positions.append(current_batch[sent][0][2][tok])

                batch_tokens.append(current_batch[sent][0][0][tok])
                #wyrzucamy macierz id dzieci batch_children_ids.append(current_batch[sent][0][1][tok])
                batch_labels.append(current_batch[sent][0][2][tok])
                batch_words.append(current_batch[sent][0][3][tok])


        #wyrzucamy macierz id dzieci batch_children_ids = seq.pad_sequences(batch_children_ids, padding='post', value = -1)
        batch_children_positions = seq.pad_sequences(batch_children_positions, padding='post', value = -1)

        sentences.append([
                            np.array(batch_tokens), 
                            #wyrzucamy macierz id dzieci numpy.array(batch_children_ids), 
                            np.array(batch_children_positions), 
                            np.array(batch_labels)
                            ,np.array(batch_words)
                        ])
           
    return sentences

In [7]:
import numpy as np
import random
import time
import os
import itertools
import pickle
import  csv
from collections import Counter, OrderedDict

from keras.preprocessing import sequence as seq

import theano
from theano import tensor as T
from theano.ifelse import ifelse
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams 

class TreeLSTM(object):  

    def __init__(self, h_dim, nc, w2v_model_path, file_with_rules, 
                 rules_emb_dim, max_phrase_length, emb_dropout_rate, h_dropout_rate, l, srng,
                load_params=None): 

        '''

        - dropout stanu ukrytego (LSTM_1)
        - dropout embeddinga (LSTM_1)
        - regularyzacja l2 (LSTM_1)
        - indywidualna obsluga lisci - struktura taka sama, macierze te same, ale uczymy: h_aggregated_0, hidden_state_0, cell_state_0, zamiast brac w te miejsca 0


        nh :: dimension of hidden state
        nc :: number of classes
        '''

        self.max_phrase_length = max_phrase_length

        w2vecs = pickle.load(open(w2v_model_path,"rb"))
        self.emb = theano.shared(w2vecs["vectors"].astype(theano.config.floatX))
        self.words2ids = w2vecs["words2ids"]

        emb_dim = w2vecs["vectors"].shape[1]
        del w2vecs

        
        r = open(file_with_rules,"r")
        rules = [x.split() for x in r.readlines()]
        r.close()
        unique_rules = set()
        for i in range(len(rules)):
            for j in range(len(rules[i])):
                unique_rules.add(rules[i][j])
                
        number_of_uniue_rules = len(unique_rules)
 
        r = 0.05

        self.rules2ids = dict(zip(unique_rules,range(number_of_uniue_rules)))
        self.emb_rules = theano.shared(r * np.random.uniform(-1,1,(number_of_uniue_rules, rules_emb_dim)).astype(theano.config.floatX))
        
   

        self.W_i = theano.shared(r * np.random.uniform(-1.0, 1.0, (emb_dim+rules_emb_dim, h_dim) ).astype(theano.config.floatX))
        self.U_i = theano.shared(r * np.random.uniform(-1.0, 1.0, (h_dim, h_dim) ).astype(theano.config.floatX))
        self.b_i = theano.shared(r * np.random.uniform(-1.0, 1.0, h_dim ).astype(theano.config.floatX))

        self.W_f = theano.shared(r * np.random.uniform(-1.0, 1.0, (emb_dim+rules_emb_dim, h_dim) ).astype(theano.config.floatX))
        self.U_f = theano.shared(r * np.random.uniform(-1.0, 1.0, (h_dim, h_dim) ).astype(theano.config.floatX))
        self.b_f = theano.shared(r * np.random.uniform(-1.0, 1.0, h_dim ).astype(theano.config.floatX))

        self.W_o = theano.shared(r * np.random.uniform(-1.0, 1.0, (emb_dim+rules_emb_dim, h_dim) ).astype(theano.config.floatX))
        self.U_o = theano.shared(r * np.random.uniform(-1.0, 1.0, (h_dim, h_dim) ).astype(theano.config.floatX))
        self.b_o = theano.shared(r * np.random.uniform(-1.0, 1.0, h_dim ).astype(theano.config.floatX))

        self.W_u = theano.shared(r * np.random.uniform(-1.0, 1.0, (emb_dim+rules_emb_dim, h_dim) ).astype(theano.config.floatX))
        self.U_u = theano.shared(r * np.random.uniform(-1.0, 1.0, (h_dim, h_dim) ).astype(theano.config.floatX))
        self.b_u = theano.shared(r * np.random.uniform(-1.0, 1.0, h_dim ).astype(theano.config.floatX))

        self.W_y   = theano.shared(r * np.random.uniform(-1.0, 1.0, (h_dim, nc)).astype(theano.config.floatX))
        self.b_y   = theano.shared(r * np.random.uniform(-1.0, 1.0, nc).astype(theano.config.floatX))




        self.h_aggregated_0 = theano.shared(r * np.random.uniform(-1.0, 1.0, h_dim ).astype(theano.config.floatX))
        self.cell_state_0 = theano.shared(r * np.random.uniform(-1.0, 1.0, h_dim ).astype(theano.config.floatX))
        self.hidden_state_0 = theano.shared(r * np.random.uniform(-1.0, 1.0, h_dim ).astype(theano.config.floatX))



        self.srng = srng
        self.h_dropout_rate = h_dropout_rate
        self.emb_dropout_rate = emb_dropout_rate
        self.l = l


        if load_params:
            for key in load_params.keys():
                if key not in ['emb', 'emb_rules', 'W_i', 'U_i', 'b_i', 'W_f', 'U_f', 'b_f', 'W_o', 'U_o', 'b_o', 'W_u', 'U_u', 'b_u', 'W_y', 'b_y', 'h_aggregated_0', 'cell_state_0', 'hidden_state_0']:
                    setattr(self, key, load_params[key])
                else:
                    setattr(self, key, theano.shared(load_params[key]))
        
        

        def one_step(word_id, rule_id, word_children_positions, y_true, k, hidden_states, cell_states, learning_rate):

            x = T.concatenate( [self.emb[word_id], self.emb_rules[rule_id] ])

            #dropout:
            mask1 = self.srng.binomial(n=1, p=1-self.emb_dropout_rate, size=(emb_dim+rules_emb_dim,), dtype='floatX')
            x = x * mask1


            tmp = word_children_positions>=0.0
            number_of_children = tmp.sum(dtype = theano.config.floatX) 
            idx_tmp = tmp.nonzero()                                                                   # indeksy realne dzieci - czyli te, gdzie nie ma -1        

            h_aggregated = ifelse(T.gt(number_of_children, 0.0), hidden_states[word_children_positions[idx_tmp]].sum(axis=0), self.h_aggregated_0)


            i = T.nnet.sigmoid(	T.dot(x, self.W_i) + T.dot(h_aggregated, self.U_i) + self.b_i)             

            o = T.nnet.sigmoid(	T.dot(x, self.W_o) + T.dot(h_aggregated, self.U_o) + self.b_o)             

            u = T.tanh(	T.dot(x, self.W_u) + T.dot(h_aggregated, self.U_u) + self.b_u)             

            f_c = ifelse(T.gt(number_of_children, 0.0), 
                (T.nnet.sigmoid( T.dot(x, self.W_f ) + T.dot(hidden_states[word_children_positions[idx_tmp]], self.U_f)  + self.b_f )*cell_states[word_children_positions[idx_tmp]]).sum(axis=0),
                T.nnet.sigmoid( T.dot(x, self.W_f ) + T.dot(self.hidden_state_0, self.U_f)  + self.b_f ) * self.cell_state_0
            )

            c = i*u + f_c

            h = o * T.tanh(c)
            #dropout:
            mask = self.srng.binomial(n=1, p=1-self.h_dropout_rate, size=(h_dim,), dtype='floatX')
            h = h * mask


            current_cell_state = cell_states[k]
            cell_states_new = T.set_subtensor(current_cell_state, c)

            current_hidden_state = hidden_states[k]
            hidden_states_new = T.set_subtensor(current_hidden_state, h)


            y_prob = T.nnet.softmax(T.dot(h,self.W_y) + self.b_y)[0]

            cross_entropy = -T.log(y_prob[y_true])						       # + norm_coefficient * l2_norm

            return cross_entropy, hidden_states_new, cell_states_new  


        y = T.vector('y',dtype=dataType)
        learning_rate = T.scalar('lr',dtype=theano.config.floatX)
        words = T.vector(dtype=dataType)
        rules = T.vector(dtype=dataType)
        children_positions = T.matrix(dtype=dataType)
        words_indexes = T.vector(dtype=dataType)

        [cross_entropy_vector, _, _] , _ = theano.scan(fn=one_step, \
                                 sequences = [words, rules, children_positions,y,words_indexes],
                                 outputs_info = [None, 
                                     theano.shared(np.zeros((self.max_phrase_length+1,h_dim), dtype = theano.config.floatX)),
                                     theano.shared(np.zeros((self.max_phrase_length+1,h_dim), dtype = theano.config.floatX))],
                                 non_sequences = learning_rate,
                                 n_steps = words.shape[0])

        cost = T.mean(cross_entropy_vector) + self.l * (self.emb_rules**2).sum() #*0.5 * self.l * ((self.W_i**2).sum()+(self.W_f**2).sum()+(self.W_o**2).sum()+(self.W_u**2).sum()+(self.W_y**2).sum()+(self.U_i**2).sum()+(self.U_f**2).sum()+(self.U_o**2).sum()+(self.U_u**2).sum())

        updates = OrderedDict([
            (self.W_i, self.W_i-learning_rate*T.grad(cost, self.W_i)),
            (self.W_f, self.W_f-learning_rate*T.grad(cost, self.W_f)),
            (self.W_o, self.W_o-learning_rate*T.grad(cost, self.W_o)),
            (self.W_u, self.W_u-learning_rate*T.grad(cost, self.W_u)),
            (self.W_y, self.W_y-learning_rate*T.grad(cost, self.W_y)),

            (self.U_i, self.U_i-learning_rate*T.grad(cost, self.U_i)),
            (self.U_f, self.U_f-learning_rate*T.grad(cost, self.U_f)),
            (self.U_o, self.U_o-learning_rate*T.grad(cost, self.U_o)),
            (self.U_u, self.U_u-learning_rate*T.grad(cost, self.U_u)),

            #(self.emb, self.emb-learning_rate*T.grad(cost, self.emb)), #SPROBOWAC TU 0.1 ZAMIAST LR, A DLA POLSKICH BEZ AKTUALIZACJI EMB
            (self.emb_rules, self.emb_rules-learning_rate*T.grad(cost, self.emb_rules)),
            (self.b_i, self.b_i-learning_rate*T.grad(cost,self.b_i)),
                        (self.b_f, self.b_f-learning_rate*T.grad(cost,self.b_f)),
                        (self.b_o, self.b_o-learning_rate*T.grad(cost,self.b_o)),
                        (self.b_u, self.b_u-learning_rate*T.grad(cost,self.b_u)),
                        (self.b_y, self.b_y-learning_rate*T.grad(cost,self.b_y)),

            (self.h_aggregated_0, self.h_aggregated_0-learning_rate*T.grad(cost,self.h_aggregated_0)),
            (self.cell_state_0, self.cell_state_0-learning_rate*T.grad(cost,self.cell_state_0)),
            (self.hidden_state_0, self.hidden_state_0-learning_rate*T.grad(cost,self.hidden_state_0))

            ])

        self.train = theano.function( inputs  = [words, rules, children_positions, y, words_indexes, learning_rate],
                                      outputs = [],
                                      updates = updates,
                                      allow_input_downcast=True,
                                      mode='FAST_RUN'
                                      )


        def one_step_classify(word_id, rule_id, word_children_positions, k, hidden_states, cell_states):

            x = T.concatenate( [self.emb[word_id], self.emb_rules[rule_id] ])

            x = (1-self.emb_dropout_rate) * x

            tmp = word_children_positions>=0.0
            number_of_children = tmp.sum(dtype = theano.config.floatX) 
            idx_tmp = tmp.nonzero()                                                                   # indeksy realne dzieci - czyli te, gdzie nie ma -1        

            h_aggregated = ifelse(T.gt(number_of_children, 0.0), hidden_states[word_children_positions[idx_tmp]].sum(axis=0), self.h_aggregated_0)

            i = T.nnet.sigmoid(	T.dot(x, self.W_i) + T.dot(h_aggregated, self.U_i) + self.b_i)             

            o = T.nnet.sigmoid(	T.dot(x, self.W_o) + T.dot(h_aggregated, self.U_o) + self.b_o)             

            u = T.tanh(	T.dot(x, self.W_u) + T.dot(h_aggregated, self.U_u) + self.b_u)             

            f_c = ifelse(T.gt(number_of_children, 0.0), 
                (T.nnet.sigmoid( T.dot(x, self.W_f ) + T.dot(hidden_states[word_children_positions[idx_tmp]], self.U_f)  + self.b_f )*cell_states[word_children_positions[idx_tmp]]).sum(axis=0),
                T.nnet.sigmoid( T.dot(x, self.W_f ) + T.dot(self.hidden_state_0, self.U_f)  + self.b_f ) * self.cell_state_0
            )

            c = i*u + f_c

            h = o * T.tanh(c)
            # podczas uczenia zerowalismy 1-dropout_rate procent wspolrzednych, wiec trzeba to 
            h = h * (1-self.h_dropout_rate)

            current_cell_state = cell_states[k]
            cell_states_new = T.set_subtensor(current_cell_state, c)

            current_hidden_state = hidden_states[k]
            hidden_states_new = T.set_subtensor(current_hidden_state, h)


            y_prob = T.nnet.softmax(T.dot(h,self.W_y) + self.b_y)[0]             

            return  y_prob, hidden_states_new, cell_states_new


        [y_probs_classify, _ , _ ], _ = theano.scan(
                 fn=one_step_classify, 
                                 sequences = [words, rules, children_positions, words_indexes],
                 outputs_info = [None,
                         theano.shared(np.zeros((self.max_phrase_length+1,h_dim), dtype = theano.config.floatX)),
                         theano.shared(np.zeros((self.max_phrase_length+1,h_dim), dtype = theano.config.floatX))])

        predictions, _ = theano.scan(lambda i: T.argmax(y_probs_classify[i]), 
                                     sequences = [words_indexes])
        
        probs, _ = theano.scan(lambda i: y_probs_classify[i], 
                                     sequences = [words_indexes])

        self.classify = theano.function(inputs=[words, rules, children_positions,words_indexes], 
                                     outputs=predictions,
                                     allow_input_downcast=True,
                                     mode='FAST_RUN' 
                                     )

        self.predict_proba = theano.function(inputs=[words, rules, children_positions,words_indexes], 
                             outputs=probs,
                             allow_input_downcast=True,
                             mode='FAST_RUN' 
                             )




In [9]:
s = {'lr':0.05,
         'nepochs':80,
         'seed':345,
         'nc':2,        # number of y classes
         'h_dim': 100,
         'h_dropout_rate': 0.5,
         'emb_dropout_rate': 0.5,
         'time_without_improvement': 10,
         'batch_size': 1,
         'w2v_DIM': "300",
         "rules_emb_dim": 30
         }  

dataType = 'int64'
  
np.random.seed(s['seed']) 

params = dict(pickle.load(open("model_params.pkl","rb")))

rnn = TreeLSTM( h_dim = s['h_dim'],
            nc = s['nc'],
        w2v_model_path = "embeddings/filtered_w2v_allwiki_nkjpfull_300.pkl",
            max_phrase_length = 60,
        emb_dropout_rate = s['emb_dropout_rate'],
        h_dropout_rate = s['h_dropout_rate'],
        l = 0.0001,
        srng = RandomStreams(12345),
        file_with_rules =  "Data/rules.txt",
        rules_emb_dim = s["rules_emb_dim"],
        load_params= params
    )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:131: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:224: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.


In [37]:
data0 = load_stanford_data4("Data/labels.txt", "Data/parents.txt","Data/tokens.txt","Data/rules.txt",w2vecs["words2ids"],True,s['batch_size'],s['nc'])

data_rules = load_stanford_data4("Data/labels.txt", "Data/parents.txt","Data/rules.txt","Data/rules.txt",rnn.rules2ids,True,s['batch_size'],s['nc'])
data_rules = [x[0] for x in data_rules]

data = [data0[i]+[data_rules[i]] for i in range(len(data0))]

In [61]:
"""
TYLE LASOW ZOSTALO UZYTYCH DO UCZENIA SIECI. ALE TO NIE ODPOWIADA ZMIENNEJ j BO TA ZMIENNA ITERUJE BO PLIKACH
A NIE TYLKO TYCH PLIKACH, KTORE ZOSTALY UZYTE
"""


t = open("Data/tokens.txt","r")
tokens = [x.split() for x in t.readlines()]
t.close()

tokens = [" ".join([x for x in line if x[:2] != '__']) for line in tokens]

numer_of_forest_used_for_training_network = np.cumsum([x != y for x,y, in zip(tokens,tokens[1:])])[40000]

numer_of_forest_used_for_training_network

5935

In [12]:
data_folder = "../Składnica-frazowa-171220/**/*.xml"

j = 1

for filename in list(glob.iglob(data_folder, recursive=True))[-1000:]:
    
    print(j)
    j = j + 1
    
    forest = ET.parse(filename)
 
    try:
    
        num_trees = number_of_trees_in_forest(forest)
        print(num_trees)

        if num_trees <= 200:
            trees = get_all_trees_randomly(forest)
        else:
            trees = []
            trees.append(get_positive_tree(forest))
            for i in range(min(num_trees-1,10000)):
                trees.append(get_random_negative_tree(forest))

        print("len(trees)=",len(trees))        

        
        
        directory = "Data/test"+str(j)
        os.makedirs(directory)
        for tree in trees:    
            write_dependency_format(transform_to_dependency_format(tree), directory)


            
            
        positive_tree = get_positive_tree(forest)
        f1=[]
        for tree in trees:
            f1.append(scores(positive_tree, tree)[2])

        #pickle.dump(f1, open(directory+"/f1.pkl","wb"))
        
        
        
        data0 = load_stanford_data4(directory+"/labels.txt", directory+"/parents.txt",directory+"/tokens.txt",directory+"/rules.txt",w2vecs["words2ids"],True,s['batch_size'],s['nc'])

        data_rules = load_stanford_data4(directory+"/labels.txt", directory+"/parents.txt",directory+"/rules.txt",directory+"/rules.txt",rnn.rules2ids,True,s['batch_size'],s['nc'])
        data_rules = [x[0] for x in data_rules]

        data = [data0[i]+[data_rules[i]] for i in range(len(data0))]

        probs = []
        for i in range(len(data)):
            probs.append(rnn.predict_proba(data[i][0],data[i][4],data[i][1], data[i][3])[-1][1])
        
        plt.scatter(f1,probs)
        plt.title(str(len(trees)))
        plt.savefig("Plots/"+str(j)+"_"+str(num_trees)+".png")
        plt.close()
        
    except:
        pass
    
    
        
print("Liczba drzew: ",len(trees))

1
4
2
48
3
60
4
4
5
6
7
3056
len(trees)= 3056
8
9
10
11
156
12
13
13573
len(trees)= 10001
14
15
6
16
6
17
835
len(trees)= 835
18
20
19
20
21
22
23
1484
len(trees)= 1484
24
25
26
1124621064
len(trees)= 10001
27
4
28
21
29
6817
len(trees)= 6817
30
6
31
13
32
4
33
4018
len(trees)= 4018
34
24
35
36
2640
len(trees)= 2640
37
38
39
18
40
85
41
9542
len(trees)= 9542
42
46
43
44
856
len(trees)= 856
45
26
46
47
11292
len(trees)= 10001
48
49
2
50
51
260
len(trees)= 260
52
10
53
54
84
55
3
56
3246
len(trees)= 3246
57
38
58
14
59
5986
len(trees)= 5986
60
18546
len(trees)= 10001
61
12168
len(trees)= 10001
62
18
63
2
64
65
66
26
67
29
68
8
69
70
1
71
72
6
73
14
74
1
75
76
77
100
78
2
79
80
81
32
82
2
83
252
len(trees)= 252
84
85
2
86
87
88
5016
len(trees)= 5016
89
488
len(trees)= 488
90
1
91
3858788
len(trees)= 10001
92
25
93
16
94
4233414
len(trees)= 10001
95
26
96
97
28
98
200313
len(trees)= 10001
99
100
11506
len(trees)= 10001
101
102
104
103
104
316
len(trees)= 316
105
22827
len(trees)= 10001
106

970
971
972
973
974
90
975
11
976
977
978
979
980
981
71
982
983
984
985
986
4401
len(trees)= 4401
987
1
988
20127
len(trees)= 10001
989
342
len(trees)= 342
990
991
264
len(trees)= 264
992
1
993
994
995
5
996
997
6
998
54
999
1000
35
Liczba drzew:  264
